In [ ]:
!pip install matplotlib pandas numpy ipywidgets

ERROR: Could not find a version that satisfies the requirement SpatialDE2 (from versions: none)
ERROR: No matching distribution found for SpatialDE2


In [ ]:
# Import of libraries for working with spatial transcriptomics data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import interact, Dropdown
w.IntSlider()

/usr/lib/python3.12/importlib/__init__.py:90: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  return _bootstrap._gcd_import(name[level:], package, level)
/usr/local/lib/python3.12/dist-packages/squidpy/gr/_utils.py:23: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('anndata')` instead.
  CAN_USE_SPARSE_ARRAY = Version(anndata.__version__) >= Version("0.11.0rc1")
/usr/local/lib/python3.12/dist-packages/anndata/__init__.py:70: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  return module_get_attr_redirect(attr_name, deprecated_mapping=_DEPRECATED)


Loading data

In [ ]:
file_path = r"C:\Users\irato\OneDrive\Робочий стіл\MUNI\IV114_ST\data_after_qc\SN048_A121573_Rep2\results_spatialDE\results.csv"
results = pd.read_csv(file_path)
print(results.head())

Graph of the tissue with the color correspond to expression level of top 10 significance genes

In [ ]:
# 1) Select top 10 genes based on q-value
top10 = results.sort_values('qval').head(10)[['g', 'l', 'qval']]

# Create maps (gene → l) and (gene → qval)
l_map = top10.set_index('g')['l'].to_dict()
qval_map = top10.set_index('g')['qval'].to_dict()

# Dropdown will contain only gene names
top10_genes = list(l_map.keys())

# 2) Function to plot a selected gene
def plot_gene(gene):
    l_value = l_map[gene]
    qval_value = qval_map[gene]
    
    title = f"{gene} | l = {l_value:.3f} | qval = {qval_value:.2e}"
    
    plt.figure(figsize=(10,3))
    plt.scatter(sample_info['x'], sample_info['y'], c=norm_expr[gene])
    plt.title(title)
    plt.axis('equal')
    plt.colorbar(ticks=[])
    plt.show()

# 3) Interactive gene selection
interact(
    plot_gene,
    gene=Dropdown(options=top10_genes, description="Gene:")
)


Volkano plot - the relation between significance and effect size. Efect size is the fraction of variance explained by spatial variation.

In [ ]:
figsize(5, 4)
plt.yscale('log')

plt.scatter(results['FSV'], results['qval'], c='black')

plt.axhline(0.05, c='black', lw=1, ls='--')

plt.gca().invert_yaxis()
plt.xlabel('Fraction spatial variance')
plt.ylabel('Adj. P-value')